## Assignment for Module 5, Training Models

In this assignment you will train different models on a given data set, and find the one that performs best

### Getting the data for the assignment (similar to the notebook from chapter 2 of Hands-On...)

In [1]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [2]:
fetch_housing_data()

In [3]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [4]:
housing = load_housing_data()

### Fix the categories in the categorical variable

In [5]:
d = {'<1H OCEAN':'LESS_1H_OCEAN',
     'INLAND':'INLAND',
     'ISLAND':'ISLAND',
     'NEAR BAY':'NEAR_BAY',
     'NEAR OCEAN':'NEAR_OCEAN'}
housing['ocean_proximity'] = housing['ocean_proximity'].map(lambda s: d[s])

### Add 2 more features

In [6]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["population_per_household"]=housing["population"] / housing["households"]

### Fix missing data

In [7]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True) 

### Create dummy variables based on the categorical variable

In [8]:
one_hot = pd.get_dummies(housing['ocean_proximity'])
housing = housing.drop('ocean_proximity', axis=1)
housing = housing.join(one_hot)

### Check the data

In [9]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 16 columns):
longitude                   20640 non-null float64
latitude                    20640 non-null float64
housing_median_age          20640 non-null float64
total_rooms                 20640 non-null float64
total_bedrooms              20640 non-null float64
population                  20640 non-null float64
households                  20640 non-null float64
median_income               20640 non-null float64
median_house_value          20640 non-null float64
rooms_per_household         20640 non-null float64
population_per_household    20640 non-null float64
INLAND                      20640 non-null uint8
ISLAND                      20640 non-null uint8
LESS_1H_OCEAN               20640 non-null uint8
NEAR_BAY                    20640 non-null uint8
NEAR_OCEAN                  20640 non-null uint8
dtypes: float64(11), uint8(5)
memory usage: 1.8 MB


# ASSIGNMENT

### Q1. Partition into train and test (1 mark)

Use train_test_split from sklearn.model_selection to partition the dataset into 70% for training and 30% for testing.

You can use the 70% for training set as both training and validation by using cross-validation.


In [28]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.3, random_state=42)

Seperating target from features

In [29]:
target = 'median_house_value'
features = list(train_set.columns)
features = [f for f in features if f != target]

In [30]:
# Train set
X_tr = train_set[features]
y_tr = train_set[[target]]

# Test set
X_te = test_set[features]
y_te = test_set[[target]]

### Q2. Polynomial transformations (1 mark)

Use PolynomialFeatures from sklearn.preprocessing to add higher degree features (degree=2).

In [36]:
import numpy as np

In [31]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
poly.fit(X_tr)
X_tr = poly.transform(X_tr)
X_te = poly.transform(X_te)

In [37]:
np.size(X_tr,1)

136

In [38]:
np.size(X_te,1)

136

You should obtain X_tr and X_te with 136 columns each, since originally you had 15 features.

With m original features, the new added polynomial features of degree 2 are: $(m^2-m)/2+m+1$. Why?

These, plus the original features gives a total of  $(m^2-m)/2+2m+1$

In [39]:
print("Original number of features: " + str(len(features)))
print("Final number of features: " + str(X_tr.shape[1]))

Original number of features: 15
Final number of features: 136


### Q3. Scaling features (1 mark)

Similarly, use StandardScaler from sklearn.preprocessing to normalize the training and testing data, using the training data

In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_tr_scaled = scaler.fit_transform(X_tr)
X_te_scaled = scaler.fit_transform(X_te)

For comparing models

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())

### Q4. Linear regression on original features (no transformations) -- benchmark only (0 marks)

Your goal is to find the model that minimizes the rmse score

In [42]:
from sklearn.linear_model import LinearRegression

lin_scores = cross_val_score(LinearRegression(),
                             train_set[features],
                             train_set[target],
                             scoring="neg_mean_squared_error",
                             cv=4)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [70142.55721218 67456.39127204 67318.3258893  70866.26065275]
Mean: 68945.88375656851


### Q5. Linear regression on transformed features -- polynomial transformation + scaling (2 marks)

Now do as in Q4 but with the transformed features (136 features)

In [48]:
transformed_lin_scores = cross_val_score(LinearRegression(),
                             X_tr_scaled,
                             y_tr,
                             scoring="neg_mean_squared_error",
                             cv=4)
transformed_lin_rmse_scores = np.sqrt(-transformed_lin_scores)
display_scores(transformed_lin_rmse_scores)

Scores: [7.62950047e+14 1.75203454e+15 1.13729475e+14 2.74127763e+14]
Mean: 725710457349905.4


If the error on the cross-validation is too high it is because the model is over-fitting. Regularization is needed.

### Q6. Ridge regression -- benchmark only (0 marks)

In [49]:
from sklearn.linear_model import Ridge

param_grid = [{'alpha': [0.001,0.01,0.1,1,10,100,1000,1000]}]
grid_search_rr = GridSearchCV(Ridge(), param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search_rr.fit(X_tr, y_tr)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.673800e-23
  overwrite_a=True).T
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.578587e-23
  overwrite_a=True).T
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number1.288330e-23
  overwrite_a=True).T
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\linear_model\ridge.py:112: LinAlgWarn

GridSearchCV(cv=3, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [50]:
print(grid_search_rr.best_params_)
print(np.sqrt(-grid_search_rr.best_score_))

{'alpha': 1000}
104583.65011940197


### Q7. Lasso regression (2 marks)

Now do the same as in Q6 but with Lasso

In [ ]:
## YOUR CODE HERE ##
## YOUR CODE HERE ##
## YOUR CODE HERE ##
## YOUR CODE HERE ##
## YOUR CODE HERE ##
## YOUR CODE HERE ##

### Q8. Elastic Net regression (2 marks)

Do the same as in 6 and 7, but now with Elastic Net. However, the grid search should be over the parameters alpha and  l1ratio. Use just 3 values for l1_ratio.

In [ ]:
## YOUR CODE HERE ##
## YOUR CODE HERE ##
## YOUR CODE HERE ##
## YOUR CODE HERE ##
## YOUR CODE HERE ##
## YOUR CODE HERE ##

Evaluating your best model on TESTING data

Choose among grid_search_rr, grid_search_lr, and grid_search_enr, the model with best performance

In [ ]:
from sklearn.metrics import mean_squared_error

final_model = grid_search.best_estimator_

y_te_estimation = final_model.predict(X_te)

final_mse = mean_squared_error(y_te, y_te_estimation)
final_rmse = np.sqrt(final_mse)
print(final_rmse)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x=y_te, y=y_te_estimation)
plt.xlim([-200000,800000])
plt.ylim([-200000,800000])
plt.show()

### Q9. Before you computed the final_rmse on the test data, what was your expected value for this quantity? Does your best model have high variance? (1 mark)

##### YOUR ANSWER HERE 